In [2]:
import sklearn
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras import layers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from utils.dataset import get_sentences, get_corpora, get_top_author_gut_idx

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # surpress tensorflow warnings

Using TensorFlow backend.


# Acquire the dataset
First time this process takes around 10 minutes with fast internet connection.
Later the texts are taken from a local mysql cache and the process is much faster (~1m)`

In [3]:
corpora = get_corpora(max_chars_per_author = 1e7, random_seed=42)

Book 001 -- Acquiring A Young Hero by Fenn, George Manville
Book 002 -- Acquiring Quicksilver: The Boy With No Skid to His Wheel by Fenn, George Manville
Book 003 -- Acquiring Featherland: How the Birds lived at Greenlawn by Fenn, George Manville
Book 004 -- Acquiring Menhardoc by Fenn, George Manville
Book 005 -- Acquiring The Man with a Shadow by Fenn, George Manville
Book 006 -- Acquiring Son Philip by Fenn, George Manville
Book 007 -- Acquiring The Queen's Scarlet: The Adventures and Misadventures of Sir Richard Frayne by Fenn, George Manville
Book 008 -- Acquiring Cursed by a Fortune by Fenn, George Manville
Book 009 -- Acquiring Fix Bay'nets: The Regiment in the Hills by Fenn, George Manville
Book 010 -- Acquiring A Life's Eclipse by Fenn, George Manville
Book 011 -- Acquiring A Little World by Fenn, George Manville
Book 012 -- Acquiring The Lost Middy: Being the Secret of the Smugglers' Gap by Fenn, George Manville
Book 013 -- Acquiring In the King's Name: The Cruise of the "Kes

Book 103 -- Acquiring My First Visit to New England, and Others (from Literary Friends and Acquaintance) by Howells, William Dean
Book 104 -- Acquiring The Kentons by Howells, William Dean
Book 105 -- Acquiring The Quality of Mercy by Howells, William Dean
Book 106 -- Acquiring A Hazard of New Fortunes — Volume 2 by Howells, William Dean
Book 107 -- Acquiring The Standard Household-Effect Company (from Literature and Life) by Howells, William Dean
Book 108 -- Acquiring Ragged Lady — Complete by Howells, William Dean
Book 109 -- Acquiring Five O'Clock Tea: Farce by Howells, William Dean
Book 110 -- Acquiring Their Silver Wedding Journey — Complete by Howells, William Dean
Book 111 -- Acquiring Their Silver Wedding Journey — Volume 2 by Howells, William Dean
Book 112 -- Acquiring The Man of Letters as a Man of Business by Howells, William Dean
Book 113 -- Acquiring The Flight of Pony Baker: A Boy's Town Story by Howells, William Dean
Book 114 -- Acquiring Literature and Life (Complete) b

Book 201 -- Acquiring A Tramp Abroad — Volume 02 by Twain, Mark
Book 202 -- Acquiring Eve's Diary, Part 1 by Twain, Mark
Book 203 -- Acquiring 1601: Conversation as it was by the Social Fireside in the Time of the Tudors by Twain, Mark
Book 204 -- Acquiring The £1,000,000 bank-note, and other new stories by Twain, Mark
Book 205 -- Acquiring The Adventures of Tom Sawyer, Part 8. by Twain, Mark
Book 206 -- Acquiring Mark Twain's Letters — Volume 4 (1886-1900) by Twain, Mark
Book 207 -- Acquiring Moments with Mark Twain by Twain, Mark
Book 208 -- Acquiring The Facts Concerning the Recent Carnival of Crime in Connecticut by Twain, Mark
Book 209 -- Acquiring The Innocents Abroad by Twain, Mark
Book 210 -- Acquiring Quotations from the Project Gutenberg Editions of the Works of Mark Twain by Twain, Mark
Book 211 -- Acquiring The Gilded Age, Part 6. by Twain, Mark
Book 212 -- Acquiring Life on the Mississippi by Twain, Mark
Book 213 -- Acquiring Life on the Mississippi, Part 3. by Twain, Mark

Book 309 -- Acquiring A Thorny Path — Volume 03 by Ebers, Georg
Book 310 -- Acquiring A Word, Only a Word — Volume 01 by Ebers, Georg
Book 311 -- Acquiring Cleopatra — Volume 02 by Ebers, Georg
Book 312 -- Acquiring The Bride of the Nile — Volume 05 by Ebers, Georg
Book 313 -- Acquiring An Egyptian Princess — Complete by Ebers, Georg
Book 314 -- Acquiring Barbara Blomberg — Volume 05 by Ebers, Georg
Book 315 -- Acquiring Serapis — Volume 06 by Ebers, Georg
Book 316 -- Acquiring A Thorny Path — Volume 07 by Ebers, Georg
Book 317 -- Acquiring Uarda : a Romance of Ancient Egypt — Complete by Ebers, Georg
Book 318 -- Acquiring Serapis — Volume 04 by Ebers, Georg
Book 319 -- Acquiring The Country Doctor by Balzac, Honoré de
Book 320 -- Acquiring Gaudissart II by Balzac, Honoré de
Book 321 -- Acquiring Modeste Mignon by Balzac, Honoré de
Book 322 -- Acquiring Christ in Flanders by Balzac, Honoré de
Book 323 -- Acquiring Ursula by Balzac, Honoré de
Book 324 -- Acquiring The Alkahest by Balzac

Book 409 -- Acquiring Red Rooney: The Last of the Crew by Ballantyne, R. M. (Robert Michael)
Book 410 -- Acquiring Fast in the Ice: Adventures in the Polar Regions by Ballantyne, R. M. (Robert Michael)
Book 411 -- Acquiring Chasing the Sun by Ballantyne, R. M. (Robert Michael)
Book 412 -- Acquiring The Garret and the Garden; Or, Low Life High Up by Ballantyne, R. M. (Robert Michael)
Book 413 -- Acquiring Hudson Bay by Ballantyne, R. M. (Robert Michael)
Book 414 -- Acquiring Philosopher Jack by Ballantyne, R. M. (Robert Michael)
Book 415 -- Acquiring The Rover of the Andes: A Tale of Adventure on South America by Ballantyne, R. M. (Robert Michael)
Book 416 -- Acquiring Digging for Gold: Adventures in California by Ballantyne, R. M. (Robert Michael)
Book 417 -- Acquiring The Big Otter by Ballantyne, R. M. (Robert Michael)
Book 418 -- Acquiring Post Haste by Ballantyne, R. M. (Robert Michael)
Book 419 -- Acquiring All But Lost: A Novel. Vol. 1 of 3 by Henty, G. A. (George Alfred)
Book 420

Book 500 -- Acquiring Captain Rogers: The Lady of the Barge and Others, Part 7. by Jacobs, W. W. (William Wymark)
Book 501 -- Acquiring Dual Control: Ship's Company, Part 8. by Jacobs, W. W. (William Wymark)
Book 502 -- Acquiring Shareholders: Deep Waters, Part 1. by Jacobs, W. W. (William Wymark)
Book 503 -- Acquiring In the Library: The Lady of the Barge and Others, Part 6. by Jacobs, W. W. (William Wymark)
Book 504 -- Acquiring The Lady of the Barge and Others, Entire Collection by Jacobs, W. W. (William Wymark)
Book 505 -- Acquiring At Sunwich Port, Part 2.: Contents: Chapters 6-10 by Jacobs, W. W. (William Wymark)
Book 506 -- Acquiring The Toll-House: Sailor's Knots, Part 7. by Jacobs, W. W. (William Wymark)
Book 507 -- Acquiring Bill's Lapse: Odd Craft, Part 4. by Jacobs, W. W. (William Wymark)
Book 508 -- Acquiring The Persecution of Bob Pretty: Odd Craft, Part 9. by Jacobs, W. W. (William Wymark)
Book 509 -- Acquiring Sailors' Knots (Entire Collection) by Jacobs, W. W. (William

Book 587 -- Acquiring Captain Cook: His Life, Voyages, and Discoveries by Kingston, William Henry Giles
Book 588 -- Acquiring Taking Tales: Instructive and Entertaining Reading by Kingston, William Henry Giles
Book 589 -- Acquiring My First Voyage to Southern Seas by Kingston, William Henry Giles
Book 590 -- Acquiring Villegagnon: A Tale of the Huguenot Persecution by Kingston, William Henry Giles
Book 591 -- Acquiring James Braithwaite, the Supercargo: The Story of his Adventures Ashore and Afloat by Kingston, William Henry Giles
Book 592 -- Acquiring The Seven Champions of Christendom by Kingston, William Henry Giles
Book 593 -- Acquiring The Loss of the Royal George by Kingston, William Henry Giles
Book 594 -- Acquiring Rob Nixon, the Old White Trader: A Tale of Central British North America by Kingston, William Henry Giles
Book 595 -- Acquiring Voyages and Travels of Count Funnibos and Baron Stilkin by Kingston, William Henry Giles
Book 596 -- Acquiring Our Sailors: Gallant Deeds o

Book 663 -- Acquiring King Henry the Fifth: Arranged for Representation at the Princess's Theatre by Shakespeare, William
Book 664 -- Acquiring The Phoenix and the Turtle by Shakespeare, William
Book 665 -- Acquiring Henry VI, Part 1 by Shakespeare, William
Book 666 -- Acquiring History of King Henry the Sixth, Second Part by Shakespeare, William
Book 667 -- Acquiring The Tragedy of Romeo and Juliet by Shakespeare, William
Book 668 -- Acquiring The Tragedy of Titus Andronicus by Shakespeare, William
Book 669 -- Acquiring The Merry Wives of Windsor by Shakespeare, William
Book 670 -- Acquiring Love's Labour's Lost by Shakespeare, William
Book 671 -- Acquiring The Tragedy of King Richard the Second by Shakespeare, William
Book 672 -- Acquiring King John by Shakespeare, William
Book 673 -- Acquiring The Winter's Tale by Shakespeare, William
Book 674 -- Acquiring Henry VI, Part 2 by Shakespeare, William
Book 675 -- Acquiring Measure for Measure by Shakespeare, William
Book 676 -- Acquiring

# Tokenize the input

In [4]:
VOCAB = 30000
t = keras.preprocessing.text.Tokenizer(
    num_words=VOCAB, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
    split=' ', char_level=False, oov_token=None, document_count=0)

In [5]:
t.fit_on_texts(list(corpora.values()))

# Convert corpora into sequences

In [6]:
corpora_seq = {}
SEQ_LEN = 100
for author in corpora:
    s = t.texts_to_sequences([corpora[author]])
    corpus_seq = np.array(s).squeeze()
    
    # reject last corp_len%seq_len words
    corp_len = corpus_seq.shape[0]
    aligned_len = corp_len - corp_len%SEQ_LEN 
    sequences = corpus_seq[:aligned_len].reshape(-1,100)
    
    corpora_seq[author] = sequences

# Make vectors X, y
X - array of shape (n_samples, seq_len)

y - array of lables (n_samples, n_classes)

In [7]:
X, y = [], []
for author, sequence in corpora_seq.items():
    X.append(sequence)
    y.append(len(sequence)*[author])
X = np.vstack(X)
y = np.concatenate(y)

In [8]:
le = sklearn.preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)
y = keras.utils.to_categorical(y)

# Train, test split

In [9]:
X, X_test, y, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.1, stratify=y)

# 1D CNN Neural Net Classifier

In [10]:
def make_model(VOCAB, BATCH_SIZE, SEQ_LEN, N_CLASSSES, LR):
    model = keras.Sequential()
    model.add(layers.Embedding(VOCAB, BATCH_SIZE, input_length=SEQ_LEN))

    model.add(layers.Conv1D(filters=250,
                            kernel_size=3,
                            padding='valid',
                            activation='relu',
                            strides=1))

    # we use max pooling:
    model.add(layers.GlobalMaxPooling1D())

    # We add a vanilla hidden layer:
    model.add(layers.Dense(250, activation='relu'))
    # model.add(layers.Dropout(0.2))
    model.add(layers.Dense(N_CLASSSES, activation='sigmoid'))
    
    optim = keras.optimizers.adam(lr=LR)
    model.compile(loss='binary_crossentropy',
                  optimizer=optim,
                  metrics=['accuracy', keras.metrics.categorical_accuracy])
    
    return model

# Hyperparameter tuning with k-fold validation

In [11]:
FOLDS = 3    
BATCH_SIZE = 64
N_CLASSSES = len(np.unique(np.argmax(y, axis=1)))
l_rates = np.logspace(1,-7,20)

sss = sklearn.model_selection.StratifiedShuffleSplit(n_splits=FOLDS, test_size=0.2, random_state=None)

models = {l: [] for l in l_rates}

for LR in l_rates:
    print(f'Testing lr={LR}')
    for train_index, test_index in sss.split(X, y):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        model = make_model(VOCAB, BATCH_SIZE, SEQ_LEN, N_CLASSSES, LR)
        model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=1,
                  validation_data=(X_val, y_val)
                 )
        models[LR].append(model)

Testing lr=10.0


/home/tomek/anaconda3/envs/cmpe255/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 273441 samples, validate on 68361 samples
Epoch 1/1


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv1d_1/convolution (defined at /home/tomek/anaconda3/envs/cmpe255/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_1242]

Function call stack:
keras_scratch_graph


# Plot val losses and val accuracies after 1 epoch

In [ ]:
k_fold_accs, k_fold_losses = {}, {}
for lr in models:
    acc = []
    loss = []
    for m in models[lr]:
        acc.append(*m.history.history['val_categorical_accuracy'])
        loss.append(*m.history.history['val_loss'])
    
    k_fold_accs[lr] = np.mean(acc)
    k_fold_losses[lr] = np.mean(loss)

In [ ]:
ax = plt.gca()
ax.set_xscale('log')
plt.xlabel("learning rate")
plt.ylabel("k-fold val loss")
plt.plot(list(k_fold_losses.keys()), list(k_fold_losses.values()))

In [ ]:
ax = plt.gca()
ax.set_xscale('log')
plt.xlabel("learning rate")
plt.ylabel("k-fold val acc")
plt.plot(list(k_fold_accs.keys()), list(k_fold_accs.values()))

# Classifier inference

In [ ]:
pred = model.predict([X_test[0:64]], batch_size=1)

In [ ]:
truth = y_test[0:64]

In [ ]:
le.inverse_transform(np.argmax(pred, axis=1))

In [ ]:
keras.metrics.categorical_accuracy(pred, truth)